# <b><span style='color:#F1A424'>AutoML - Regression - Fish Weight Prediction - Load Models </span> </b>

### Disclaimer
The sample code (“Sample Code”) provided is not covered by any Teradata agreements. Please be aware that Teradata has no control over the model responses to such sample code and such response may vary. The use of the model by Teradata is strictly for demonstration purposes and does not constitute any form of certification or endorsement. The sample code is provided “AS IS” and any express or implied warranties, including the implied warranties of merchantability and fitness for a particular purpose, are disclaimed. In no event shall Teradata be liable for any direct, indirect, incidental, special, exemplary, or consequential damages (including, but not limited to, procurement of substitute goods or services; loss of use, data, or profits; or business interruption) sustained by you or a third party, however caused and on any theory of liability, whether in contract, strict liability, or tort arising in any way out of the use of this sample code, even if advised of the possibility of such damage.

## <b> Problem overview:</b>
    

**Dataset used : Fish Weight Dataset**

**Features**:

- `Species`: Species name of fish
- `Length1`: Vertical length in cm
- `Length2`: Diagonal length in cm
- `Length3`: Cross length in cm
- `Height`: Height in cm
- `Width`: Diagonal width in cm

**Target Variable**:

- `Weight`: Weight of fish in gram

        
**Objective**:

The primary objective is typically to build a model that can accurately predict weight of fish in gram.

**Usecase**:

Here, we will use AutoML(Automated Machine Learning) functionality to automate the entire process of developing a predictive model. It will perform feature exploration, feature engineering, data preparation, model training and evaluation on dataset in auto run and at end we will get leaderboard containined different models along with their performance. Model will also have rank associated with them which indicates which is best performing model for given data followed by other models.

In [1]:
# Importing AutoML from teradataml
from teradataml import AutoML, AutoRegressor

In [2]:
# Importing other important libraries
import getpass
from teradataml import create_context, remove_context
from teradataml import DataFrame
from teradataml import load_example_data

In [3]:
# Create the connection.
host = getpass.getpass("Host: ")
username = getpass.getpass("Username: ")
password = getpass.getpass("Password: ")

con = create_context(host=host, username=username, password=password)

Host:  ········
Username:  ········
Password:  ········


## <b><span style='color:#F1A424'>| 1.</span> Loading Dataset </b>

In [4]:
# Loading dataset from example data collection
load_example_data('teradataml','fish')

In [5]:
# Fetching in teradata dataframe
fish_df = DataFrame("fish")

In [6]:
# first 5 rows
fish_df.head()

Species,Weight,Length1,Length2,Length3,Height,Width
Bream,450,26.8,29.7,34.7,13.6024,4.9274
Bream,450,27.6,30.0,35.1,14.0049,4.8438
Bream,500,28.5,30.7,36.2,14.2266,4.9594
Bream,390,27.6,30.0,35.0,12.67,4.69
Bream,290,24.0,26.3,31.2,12.48,4.3056
Bream,242,23.2,25.4,30.0,11.52,4.02
Bream,363,26.3,29.0,33.5,12.73,4.4555
Bream,500,26.8,29.7,34.5,14.1795,5.2785
Bream,430,26.5,29.0,34.0,12.444,5.134
Bream,340,23.9,26.5,31.1,12.3778,4.6961


In [7]:
# fish_df dataset
fish_df.shape

(159, 7)

In [8]:
# Performing sampling to get 80% for trainning and 20% for testing
fish_df_sample = fish_df.sample(frac = [0.8, 0.2])

In [9]:
fish_df_sample.head()

Species,Weight,Length1,Length2,Length3,Height,Width,sampleid
Bream,290,24.0,26.3,31.2,12.48,4.3056,1
Bream,500,29.1,31.5,36.4,13.7592,4.368,1
Bream,340,29.5,32.0,37.3,13.9129,5.0728,1
Bream,500,28.7,31.0,36.2,14.3714,4.8146,1
Bream,430,26.5,29.0,34.0,12.444,5.134,2
Bream,340,23.9,26.5,31.1,12.3778,4.6961,1
Bream,500,26.8,29.7,34.5,14.1795,5.2785,1
Bream,475,28.4,31.0,36.2,14.2628,5.1042,1
Bream,363,26.3,29.0,33.5,12.73,4.4555,1
Bream,450,26.8,29.7,34.7,13.6024,4.9274,1


In [10]:
# Fetching train and test data
fish_df_train= fish_df_sample[fish_df_sample['sampleid'] == 1].drop('sampleid', axis=1)
fish_df_test = fish_df_sample[fish_df_sample['sampleid'] == 2].drop('sampleid', axis=1)

In [11]:
# train data shape
fish_df_train.shape

(127, 7)

In [12]:
# test data shape
fish_df_test.shape

(32, 7)

In [13]:
#train dataset
fish_df_train.head()

Species,Weight,Length1,Length2,Length3,Height,Width
Smelt,12,12.1,13.0,13.8,2.277,1.2558
Smelt,9,10.4,11.0,12.0,2.196,1.38
Smelt,7,10.0,10.5,11.6,1.972,1.16
Pike,1650,59.0,63.4,68.0,10.812,7.48
Pike,1600,56.0,60.0,64.0,9.6,6.144
Pike,567,43.2,46.0,48.7,7.792,4.87
Pike,430,35.5,38.0,40.5,7.29,4.5765
Smelt,19,13.8,15.0,16.2,2.9322,1.8792
Smelt,13,11.7,12.4,13.5,2.43,1.269
Smelt,10,11.3,11.8,13.1,2.2139,1.2838


In [14]:
# test dataset
fish_df_test.head()

Species,Weight,Length1,Length2,Length3,Height,Width
Roach,390,29.5,31.7,35.0,9.485,5.355
Roach,290,24.0,26.0,29.2,8.8768,4.4968
Roach,272,25.0,27.0,30.6,8.568,4.7736
Smelt,8,10.8,11.3,12.6,1.9782,1.2852
Smelt,19,13.2,14.3,15.2,2.8728,2.0672
Bream,430,26.5,29.0,34.0,12.444,5.134
Smelt,12,11.5,12.2,13.4,2.0904,1.3936
Roach,169,22.0,24.0,27.2,7.5344,3.8352
Roach,40,12.9,14.1,16.2,4.1472,2.268
Parkki,300,24.0,26.0,29.0,11.368,4.234


## <b><span style='color:#F1A424'>| 2.</span> Loading Deployed Models - 'fish_models_3_5' </b>

In [15]:
# Creating AutoRegressor instance

aml = AutoRegressor()

In [16]:
# Loading models

loaded_models = aml.load('fish_models_3_5')

In [17]:
loaded_models

,RANK,MODEL_ID,FEATURE_SELECTION,MAE,MSE,MSLE,MAPE,MPE,RMSE,RMSLE,ME,R2,EV,MPD,MGD,DATA_TABLE,ADJUSTED_R2
0,1,XGBOOST_4,pca,97.400419,20220.082367,0.089251,24.918444,-5.127184,142.197336,0.298750,428.663688,0.861468,0.899440,26.383080,0.077946,ml__Weight_pca_1768972310783236,0.856851
1,2,XGBOOST_2,rfe,168.997647,50368.334722,0.303665,60.241476,-33.259595,224.428908,0.551058,602.872469,0.654917,0.715779,79.088501,0.242467,ml__Weight_rfe_1768972204017098,0.637370


## <b><span style='color:#F1A424'>| 3.</span> Get Hyperparameter for Loaded Model : </b>

In [18]:
aml.model_hyperparameters(rank=1)

{'response_column': 'Weight',
 'model_type': 'Regression',
 'column_sampling': 0.6,
 'min_impurity': 0.2,
 'lambda1': 0.01,
 'shrinkage_factor': 0.3,
 'max_depth': 10,
 'min_node_size': 1,
 'iter_num': 20,
 'num_boosted_trees': 10,
 'seed': 42}

In [19]:
aml.model_hyperparameters(rank=2)

{'response_column': 'Weight',
 'model_type': 'Regression',
 'column_sampling': 1,
 'min_impurity': 0.0,
 'lambda1': 0.01,
 'shrinkage_factor': 0.5,
 'max_depth': 5,
 'min_node_size': 2,
 'iter_num': 30,
 'num_boosted_trees': 10,
 'seed': 42}

## <b><span style='color:#F1A424'>| 4.</span> Generate Prediction using loaded models: </b>

In [20]:
prediction = aml.predict(fish_df_test)

In [22]:
prediction

automl_id,Prediction,Confidence_Lower,Confidence_upper,Weight
68,785.8978398000002,537.7856126055613,1034.010066994439,900
72,613.7667591000001,421.7934463416043,805.7400718583958,700
10,538.4007919,377.0491627620873,699.7524210379127,600
76,222.35807790000004,146.16827141659726,298.5478843834028,180
84,154.4780247,83.01753878566946,225.93851061433054,135
4,821.3363123000003,564.2771162865176,1078.3955083134829,1550
6,416.4260809999999,276.51486480167915,556.3372971983206,430
18,593.8516425,409.2081595180582,778.4951254819418,575
26,475.2733388999998,328.6462443827427,621.900433417257,500
60,814.7039561000004,558.3586689135209,1071.0492432864799,1000


In [23]:
prediction.shape

(32, 5)

### Prediction with `preserve_columns`

In [24]:
prediction = aml.predict(fish_df_test, rank=2, preserve_columns=True)

In [25]:
prediction

automl_id,Prediction,Confidence_Lower,Confidence_upper,Weight,r_Species_2,r_Length1,r_Length2,r_Length3,r_Height,r_Width
10,517.7625846,271.53073564373676,763.9944335562632,600,0,0.3861274942904212,0.40544024741321016,0.5972097699319804,1.3558325507761604,0.42096232298684794
4,635.9867846999999,350.90155120127673,921.0720181987231,1550,0,3.22844301015415,3.190603870467645,3.0423976570893156,0.1329929837976402,0.982158077053348
8,258.0821993,151.81495743003865,364.3494411699614,273,0,-0.29773789298506215,-0.29085065835039836,-0.2421830868533739,0.4728228948360455,-0.17114186965133732
12,277.34275429999997,148.55374284359286,406.13176575640705,300,0,-0.19088392622326786,-0.19138052895559715,-0.15094473285496585,0.5367693834723049,-0.11924337204962657
24,495.2180647,263.74215801187586,726.6939713881242,390,0,0.3968128909666008,0.37559920859476975,0.3964853911354824,0.10672924739346233,0.5271812480783498
28,246.8722256,143.39485816579054,350.3495930342095,169,0,-0.40459185974685646,-0.39032078774519957,-0.3151737700521004,-0.33874939808470533,-0.3492113814225409
60,665.0560035000001,360.4093551455794,969.7026518544208,1000,1,1.6363189054034148,1.599081800150825,1.454850297517016,0.7927380422705874,1.8193385083662788
68,671.2049934,365.7124292667553,976.6975575332448,900,1,1.198217641680058,1.20120128257162,1.0807730461235427,0.6194430580663256,1.6055166982472302
72,640.6620033000002,354.47095174151053,926.8530548584897,700,1,0.9310827247755723,0.9027908943872163,0.7979341487284777,0.4150426747468547,1.0517020638396404
76,242.56075029999994,142.11405422669748,343.0074463733024,180,1,-0.29773789298506215,-0.29085065835039836,-0.379040617850986,-0.5888030066841341,-0.4366891823800912


## <b><span style='color:#F1A424'>| 7.</span> Get original data with id column added using 'get_raw_data_with_id' </b> 

### testing data

In [26]:
test_data_with_id=aml.get_raw_data_with_id(fish_df_test)

In [27]:
test_data_with_id

Species,Weight,Length1,Length2,Length3,Height,Width,automl_id
Bream,242,23.2,25.4,30.0,11.52,4.02,14
Bream,975,37.4,41.0,45.9,18.6354,6.7473,22
Bream,500,28.5,30.7,36.2,14.2266,4.9594,26
Bream,600,29.4,32.0,37.2,15.438,5.58,30
Pike,1550,56.0,60.0,64.0,9.6,6.144,4
Parkki,273,23.0,25.0,28.0,11.088,4.144,8
Parkki,300,24.0,26.0,29.0,11.368,4.234,12
Parkki,140,19.0,20.7,23.2,8.5376,3.2944,16
Roach,40,12.9,14.1,16.2,4.1472,2.268,20
Roach,390,29.5,31.7,35.0,9.485,5.355,24


### Get merged testing data with prediction

In [28]:
merge_df=test_data_with_id.merge(right = prediction, how = "inner", on = "automl_id=automl_id", lsuffix = "test_df", rsuffix = "prediction_df")

In [29]:
merge_df

Species,Weight_test_df,Weight_prediction_df,Length1,Length2,Length3,Height,Width,automl_id_test_df,automl_id_prediction_df,Prediction,Confidence_Lower,Confidence_upper,r_Species_2,r_Length1,r_Length2,r_Length3,r_Height,r_Width
Perch,700,700,34.5,37.0,39.4,10.835,6.2646,72,72,640.6620033000002,354.47095174151053,926.8530548584897,1,0.9310827247755723,0.9027908943872163,0.7979341487284777,0.4150426747468547,1.0517020638396404
Perch,135,135,20.0,22.0,23.5,5.875,3.525,84,84,214.38455040000017,117.30367978240027,311.4654210176001,1,-0.618299793270445,-0.589261046534802,-0.6527556798462101,-0.7177236953811635,-0.5280882031564376
Perch,115,115,19.0,21.0,22.5,5.9175,3.3075,48,48,191.46951230000013,95.87823761148951,287.06078698851076,1,-0.7251537600322394,-0.6887311759296033,-0.7439940338446182,-0.7080175319274455,-0.6535095723605719
Perch,150,150,20.5,22.5,24.0,6.792,3.624,80,80,219.63436110000015,122.5279757153949,316.7407464846054,1,-0.5648728098895479,-0.5395259818374014,-0.607136502847006,-0.5082989450974151,-0.4709998557945555
Perch,120,120,20.0,22.0,23.5,6.11,3.4075,56,56,213.10722050000018,115.51471934595408,310.6997216540463,1,-0.618299793270445,-0.589261046534802,-0.6527556798462101,-0.6640543209900175,-0.5958445750253376
Perch,110,110,19.0,21.0,22.5,5.6925,3.555,52,52,192.7797727000001,95.66763676465389,289.89190863534634,1,-0.7251537600322394,-0.6887311759296033,-0.7439940338446182,-0.7594031031530111,-0.5107887039558672
Bream,500,500,28.5,30.7,36.2,14.2266,4.9594,26,26,491.07104229999993,251.6505124264566,730.4915721735433,0,0.2899589242048065,0.27612907919996854,0.5059714159335723,1.1896173563851984,0.2990585186201626
Bream,575,575,31.3,34.0,39.5,15.1285,5.5695,18,18,553.4239512,290.4104699840097,816.4374324159903,0,0.5891500311378306,0.6043805062028126,0.8070579841283186,1.3955935638889203,0.6508726673624269
Bream,600,600,29.4,32.0,37.2,14.9544,5.1708,10,10,517.7625846,271.53073564373676,763.9944335562632,0,0.3861274942904212,0.40544024741321016,0.5972097699319804,1.3558325507761604,0.42096232298684794
Bream,430,430,26.5,29.0,34.0,12.444,5.134,6,6,439.3230066,217.82305851778636,660.8229546822135,0,0.07625099068121788,0.10702985922880652,0.30524703713707435,0.7825066040887864,0.39974160396748204


In [30]:
merge_df.shape

(32, 19)

## <b><span style='color:#F1A424'>| 8.</span> Generate Performance Metrics : </b>

In [31]:
# Fetching performance metrics on test data
performance_metrics = aml.evaluate(fish_df_test)

In [32]:
performance_metrics


############ result Output ############

         MAE           MSE      MSLE       MAPE        MPE        RMSE     RMSLE          ME        R2        EV        MPD       MGD
0  66.215425  20427.716944  0.269352  54.326009 -39.557246  142.925564  0.518992  728.663688  0.832356  0.844966  28.339704  0.197423


## <b><span style='color:#F1A424'>| 9.</span> Get transformed data with 'get_transformed_data'

In [34]:
trans_data=aml.get_transformed_data(fish_df_test)

In [35]:
trans_data

{'rfe_test':    Weight  r_Species_2  automl_id  r_Length1  r_Length2  r_Length3  r_Height   r_Width
 0     600            0         10   0.386127   0.405440   0.597210  1.355833  0.420962
 1    1550            0          4   3.228443   3.190604   3.042398  0.132993  0.982158
 2     273            0          8  -0.297738  -0.290851  -0.242183  0.472823 -0.171142
 3     300            0         12  -0.190884  -0.191381  -0.150945  0.536769 -0.119243
 4     390            0         24   0.396813   0.375599   0.396485  0.106729  0.527181
 5     169            0         28  -0.404592  -0.390321  -0.315174 -0.338749 -0.349211
 6    1000            1         60   1.636319   1.599082   1.454850  0.792738  1.819339
 7     900            1         68   1.198218   1.201201   1.080773  0.619443  1.605517
 8     700            1         72   0.931083   0.902791   0.797934  0.415043  1.051702
 9     180            1         76  -0.297738  -0.290851  -0.379041 -0.588803 -0.436689,
 'pca_test':    aut

In [36]:
trans_data['pca_test']

automl_id,col_0,col_1,col_2,col_3,Weight
68,2.540812116992803,-0.6811569854939806,0.6784795392385081,0.09762024655980649,900
72,1.824647498512133,-0.620043104982375,0.5506496664473596,-0.02205068770558548,700
10,1.4365821437699917,1.1554196336492086,-0.12728869983366053,-0.18965030104110303,600
76,-0.8843961728159813,-0.49582095661353104,0.384475072022771,-0.23407096588042564,180
84,-1.3852915146019693,-0.39160876303103553,0.49261730239433177,-0.1971339215165966,135
4,4.858308689122894,-2.094535428259602,-1.8777141888769353,-0.4025189736998666,1550
6,0.7880611803596032,0.9416042040640207,-0.06282149063868536,-0.11174568954106712,430
18,1.8352429476712684,1.0447169767503781,-0.13317441233792854,-0.15633576714231037,575
26,1.1715648750328254,1.1124512241383206,-0.15228160521147016,-0.2019093707500797,500
60,3.2592093682386802,-0.8276982389513375,0.5731242564593729,0.07811575230337141,1000


In [37]:
trans_data['rfe_test']

Weight,r_Species_2,automl_id,r_Length1,r_Length2,r_Length3,r_Height,r_Width
600,0,10,0.3861274942904212,0.40544024741321016,0.5972097699319804,1.3558325507761604,0.42096232298684794
1550,0,4,3.22844301015415,3.190603870467645,3.0423976570893156,0.1329929837976402,0.982158077053348
273,0,8,-0.29773789298506215,-0.29085065835039836,-0.2421830868533739,0.4728228948360455,-0.17114186965133732
300,0,12,-0.19088392622326786,-0.19138052895559715,-0.15094473285496585,0.5367693834723049,-0.11924337204962657
390,0,24,0.3968128909666008,0.37559920859476975,0.3964853911354824,0.10672924739346233,0.5271812480783498
169,0,28,-0.40459185974685646,-0.39032078774519957,-0.3151737700521004,-0.33874939808470533,-0.3492113814225409
1000,1,60,1.6363189054034148,1.599081800150825,1.454850297517016,0.7927380422705874,1.8193385083662788
900,1,68,1.198217641680058,1.20120128257162,1.0807730461235427,0.6194430580663256,1.6055166982472302
700,1,72,0.9310827247755723,0.9027908943872163,0.7979341487284777,0.4150426747468547,1.0517020638396404
180,1,76,-0.29773789298506215,-0.29085065835039836,-0.379040617850986,-0.5888030066841341,-0.4366891823800912


## <b><span style='color:#F1A424'>| 8.</span> Remove deployed models

In [38]:
aml.remove_saved_models('fish_models_3_5')

In [39]:
remove_context()

True